In [1]:
from utils import RNN, device, SampleMetroDataset, ForecastMetroDataset
import torch
from torch.utils.data import DataLoader


In [5]:
# Exo 2

# Nombre de stations utilisé (dim de output)
CLASSES = 10
#Longueur des séquences
LENGTH = 20
# Dimension de l'entrée (1 (in) ou 2 (in/out))
DIM_INPUT = 2
#Taille du batch
BATCH_SIZE = 32

PATH = "../data/"
matrix_train, matrix_test = torch.load(open(PATH+"hzdataset.pch","rb"),  weights_only=True)
# matrix_train, matrix_test = torch.load(open("/home/wujinyi/sorbonnefile/AMAL/student_tp3/data/hzdataset.pch","rb"), weights_only=True)

ds_train = SampleMetroDataset(matrix_train[:, :, :CLASSES, :DIM_INPUT], length = LENGTH)
ds_test = SampleMetroDataset(matrix_test[:, :, :CLASSES, :DIM_INPUT], length = LENGTH, stations_max = ds_train.stations_max)
data_train = DataLoader(ds_train,batch_size=BATCH_SIZE,shuffle=True)
data_test = DataLoader(ds_test, batch_size=BATCH_SIZE,shuffle=False)
print(matrix_train.size(), matrix_test.size())
for x_batch, y_batch in data_train:
    print(x_batch.size(), y_batch.size())
    break

torch.Size([18, 73, 80, 2]) torch.Size([7, 73, 80, 2])
torch.Size([32, 20, 2]) torch.Size([32])


In [6]:
#  TODO:  Question 2 : prédiction de la ville correspondant à une séquence
DIM_HIDDEN = 64
EPOCHS = 20

model = RNN(DIM_INPUT, DIM_HIDDEN, CLASSES).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for x_batch, y_batch in data_train:
        batch_size = y_batch.size(0)
        x_batch = x_batch.transpose(0,1)
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        h_0 = torch.zeros(batch_size,DIM_HIDDEN).to(device)

        optimizer.zero_grad()
        h_seq = model(x_batch, h_0)
        y_pred = model.decode(h_seq[-1])
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        total += batch_size
        total_loss += loss.item()
        correct += (torch.argmax(y_pred, dim=1) == y_batch).sum().item()
    
    avg_loss = total_loss / total
    accuracy = correct/total
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, Accuracy: {accuracy*100:.2f}%")

Epoch 1, Loss: 0.0684, Accuracy: 19.91%
Epoch 2, Loss: 0.0621, Accuracy: 29.69%
Epoch 3, Loss: 0.0606, Accuracy: 29.21%
Epoch 4, Loss: 0.0599, Accuracy: 28.89%
Epoch 5, Loss: 0.0596, Accuracy: 30.36%
Epoch 6, Loss: 0.0574, Accuracy: 35.23%
Epoch 7, Loss: 0.0568, Accuracy: 38.95%
Epoch 8, Loss: 0.0560, Accuracy: 42.20%
Epoch 9, Loss: 0.0556, Accuracy: 42.32%
Epoch 10, Loss: 0.0551, Accuracy: 43.61%
Epoch 11, Loss: 0.0550, Accuracy: 44.06%
Epoch 12, Loss: 0.0546, Accuracy: 45.15%
Epoch 13, Loss: 0.0542, Accuracy: 46.66%
Epoch 14, Loss: 0.0539, Accuracy: 46.67%
Epoch 15, Loss: 0.0538, Accuracy: 46.72%
Epoch 16, Loss: 0.0534, Accuracy: 47.92%
Epoch 17, Loss: 0.0533, Accuracy: 48.14%
Epoch 18, Loss: 0.0534, Accuracy: 48.40%
Epoch 19, Loss: 0.0530, Accuracy: 50.16%
Epoch 20, Loss: 0.0528, Accuracy: 50.21%


In [2]:
# Exo 3

# Nombre de stations utilisé
CLASSES = 10
#Longueur des séquences
LENGTH = 20
# Dimension de l'entrée (1 (in) ou 2 (in/out))
DIM_INPUT = 2
#Taille du batch
BATCH_SIZE = 32

PATH = "../data/"

matrix_train, matrix_test = torch.load(open(PATH+"hzdataset.pch", "rb"), weights_only=True)
ds_train = ForecastMetroDataset(
    matrix_train[:, :, :CLASSES, :DIM_INPUT], length=LENGTH)
ds_test = ForecastMetroDataset(
    matrix_test[:, :, :CLASSES, :DIM_INPUT], length=LENGTH, stations_max=ds_train.stations_max)
data_train = DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True)
data_test = DataLoader(ds_test, batch_size=BATCH_SIZE, shuffle=False) 


In [4]:
DIM_HIDDEN = 64
EPOCHS = 20 

model_m2m = RNN(DIM_INPUT, DIM_HIDDEN, DIM_INPUT).to(device)
optimizer = torch.optim.Adam(model_m2m.parameters(), lr = 0.001)
criterion = torch.nn.MSELoss()

for epoch in range(EPOCHS):
    model_m2m.train()
    total_loss = 0
    total = 0

    for x_batch, y_batch in data_train:
        batch_size = y_batch.size(0)
        x_batch = x_batch.transpose(0,1)
        y_batch = y_batch.transpose(0,1)
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        h_0 = torch.zeros(size=(batch_size, CLASSES, DIM_HIDDEN)).to(device)

        h_seq = model_m2m(x_batch, h_0)
        y_pred = model_m2m.decode(h_seq)
        loss = criterion(y_pred, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += batch_size
        total_loss += loss.item()
    
    avg_loss = total_loss / total
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")
    

Epoch 1, Loss: 0.0011
Epoch 2, Loss: 0.0007
Epoch 3, Loss: 0.0005
Epoch 4, Loss: 0.0004
Epoch 5, Loss: 0.0004
Epoch 6, Loss: 0.0003
Epoch 7, Loss: 0.0003
Epoch 8, Loss: 0.0003
Epoch 9, Loss: 0.0003
Epoch 10, Loss: 0.0003
Epoch 11, Loss: 0.0003
Epoch 12, Loss: 0.0002
Epoch 13, Loss: 0.0002
Epoch 14, Loss: 0.0002
Epoch 15, Loss: 0.0002
Epoch 16, Loss: 0.0002
Epoch 17, Loss: 0.0002
Epoch 18, Loss: 0.0002
Epoch 19, Loss: 0.0002
Epoch 20, Loss: 0.0002
